In [1]:
from datetime import datetime
from meteostat import Daily
import ssl

# Set SSL context.
ssl._create_default_https_context = ssl._create_unverified_context

# Set time period.
start = datetime(2019, 1, 1)
end = datetime(2023, 12, 30)

# Get daily data.
data = Daily('10381', start, end)
data = data.fetch()

# Drop unnecessary columns and rename.
data = data.drop(columns=['tmin', 'tmax', 'wdir', 'wspd', 'pres', 'wpgt', 'tsun'])
data = data.rename(columns={'time': 'date', 'tavg': 'temperature', 'prcp': 'precipitation', 'snow': 'snow(mm)'})

# Export the frame to a csv file for later processing.
data.to_csv('../data/weather.csv')